# Dicom Viewer

In [46]:
# import all the needed modules
import os
from glob import glob

In [47]:

# check the current working direcotry
cwd = os.getcwd()
print("current working dir:", cwd)

current working dir: /home/ytx/myProjects/TensorflowV2/Dicom


In [48]:
# set the dicom path
dicom_dir =  "I:/dataset/LeiSang/bleed_2020-01-17_15-54-43/BleedingDataDCM/ZA-006_000"
print("one seq. dicom data dir:", dicom_dir)
# list the dicom files in the folder
print(os.path.join(dicom_dir, "*.DCM"))

label_in_marked_paths =  glob(os.path.join(dicom_dir, "*.raw"))
# print("list all the files in the dicom dir:", files_in_dicom_dir)
# print("the first dicom slice:", files_in_dicom_paths)

# sorted_label_paths =  sorted(label_in_marked_paths)                               

# print()
# print("print sorted raw file path :", sorted_label_paths[0], sorted_label_paths[1],"..." ,sorted_label_paths[-1])
# # print("the total number of RAWs:", len(sorted_label_paths))

# mask_path =   [p for p in sorted_label_paths if "marked" in p]
# print("actual label path:", mask_path)




    
    

one seq. dicom data dir: I:/dataset/LeiSang/bleed_2020-01-17_15-54-43/BleedingDataDCM/ZA-006_000
I:/dataset/LeiSang/bleed_2020-01-17_15-54-43/BleedingDataDCM/ZA-006_000/*.DCM


## ---> visualize dicom 


In [49]:
!pip install pydicom

In [50]:
import numpy as np
import matplotlib.pyplot as plt
# import os, glob
import pydicom
import pylab as pl
import sys
import matplotlib.path as mplPath

# class IndexTracker(object):
#     def __init__(self, ax, X):
#         self.ax = ax
#         ax.set_title('Scroll to Navigate through the DICOM Image Slices')

#         self.X = X
#         rows, cols, self.slices = X.shape
#         self.ind = self.slices//2

#         self.im = ax.imshow(self.X[:, :, self.ind])
#         self.update()

#     def onscroll(self, event):
#         print("%s %s" % (event.button, event.step))
#         if event.button == 'up':
#             self.ind = (self.ind + 1) % self.slices
#         else:
#             self.ind = (self.ind - 1) % self.slices
#         self.update()

#     def update(self):
#         self.im.set_data(self.X[:, :, self.ind])
#         ax.set_ylabel('Slice Number: %s' % self.ind)
#         self.im.axes.figure.canvas.draw()

# fig, ax = plt.subplots(1,1)

# os.system("tree C:/Users/Dicom_ROI")

# plots = []

# for (index, f) in enumerate(files_in_dicom_paths):

#     filename = f.split("/")[-1].split("\\")[-1]
 
#     ds = pydicom.dcmread(f)
#     pix = ds.pixel_array
#     if index==0:
#         print("each dicom path {}: at{}".format(f, index))
#         print("each dicom info:")
#         print("each file name:", filename)
#         print("each dicom shape:", pix.shape)
#         print("each min:{}, max:{}".format(np.min(pix), np.max(pix)))
#     pix = pix*1+(-1024)
#     plots.append(pix)

# y = np.dstack(plots)

# tracker = IndexTracker(ax, y)

# fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
# plt.show()

In [65]:
# !pip install ipywidgets # only need to run once if not installed

# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from IPython.display import display
th_mode = ["default", "LeiSang"]

train_or_val = ["train", "val"]

print()
#/media/ytx/Japan_Deep_Data/dataset/LeiSang/bleed_2020-01-17_15-54-43/BleedingDataDCM
# dicom_root = "/media/ytx/Japan_Deep_Data/dataset/LeiSang/bleed_2020-01-17_15-54-43/BleedingDataDCM"
# dicom_dirs =  os.listdir(dicom_root)

def get_dcm_mask_paths(dicom_root, dcm_dir, mode):
    dcm_dir_full = os.path.join(dicom_root, dcm_dir, "*.DCM")
    raw_dir_full =  os.path.join(dicom_root, dcm_dir, "*.raw")
    print("dcm_dir_full_path:",dcm_dir_full)
    print("raw_dir_full_path:",raw_dir_full)
    print()
    files_in_dicom_paths =  glob(dcm_dir_full)
    sorted_dicom_paths =  sorted(files_in_dicom_paths)
    print("sorted dicom paths :", sorted_dicom_paths[0], sorted_dicom_paths[1],"..." ,sorted_dicom_paths[-1])
    print("the total number of DCMS:", len(files_in_dicom_paths))
    print()
    label_in_marked_paths =  glob(raw_dir_full)
    mask_path =   [p for p in label_in_marked_paths if "marked" in p]
    print("actual label path:", mask_path)
    print("the total number of labels:", len(mask_path))
    print()

    # for displaying
    ind =  widgets.IntSlider(value=1, min=1, max= len(files_in_dicom_paths))
    if mode == "LeiSang":
        LB =  widgets.IntSlider(value=20, min=-2049, max= 4000)
        HB =  widgets.IntSlider(value=380, min=-2049, max= 4000)
    elif mode =="default":
        LB =  widgets.IntSlider(value=-130, min=-2049, max= 4000)
        HB =  widgets.IntSlider(value=250, min=-2049, max= 4000)
    
    
    def display(index, LW, HW):
       
        
        print("dcm path------->:", files_in_dicom_paths[index-1])
        ds = pydicom.dcmread(files_in_dicom_paths[index-1])
    #     print("DCM info:", ds)
    #         print("Filename:", filename)

        # get dcm pixel content:
        pix = ds.pixel_array
        print("Shape info: ---------->", pix.shape )
        print("raw DCM content range [{}, {}]".format(np.min(pix), np.max(pix)))

        # threshold   # lei-sang wa center 200, window, 360, slides 60, 380
        pix[pix<LW] = LW # low boundary , if < LW , set to LW
        pix[pix>HW] = HW # high boundary, if > Hw, Set to 255
        print("after windowed DCM content range [{}, {}]".format(np.min(pix), np.max(pix)))

        # normalize to [0, 255] and plot
        plt.imshow(pix, cmap='gray',vmin=0, vmax=255)
    
    interact(display, index=ind, LW=LB, HW=HB);
    return files_in_dicom_paths, mask_path






sub_dicom_dirs2 = None

def get_sub_DCM_folders(train_or_val):
    if train_or_val == "train":
        dicom_root = "/media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train"
    else:
        dicom_root = "/media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/val"
    sub_dicom_dirs = os.listdir(dicom_root)
    
#     display(sub_dicom_dirs)
    interact(get_dcm_mask_paths,dicom_root=dicom_root, dcm_dir= sub_dicom_dirs, mode=th_mode)
#     return sub_dicom_dirs

def dcm_viewer(train_or_val):
    
    interact(get_sub_DCM_folders,train_or_val=train_or_val)
    

  

    
    
    
    

#     interact(display, index=ind, LW=LB, HW=HB);

# interact(get_sub_DCM_folders, train_or_val=train_or_val) 
# print(sub_dicom_dirs2)
dcm_viewer(train_or_val=train_or_val);

interactive(children=(Dropdown(description='train_or_val', options=('train', 'val'), value='train'), Output())…